### Generate Architecture Diagram
- Diagram is generated based on identified realtionships and entities
- A Diagramming API is used for diagram generation because of pointers

In [1]:
# Install necessary libraries
!pip install diagrams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.5/875.5 KB 34.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import os
import json

from diagrams import Diagram
from diagrams.c4 import Person, Container, Database, System, SystemBoundary, Relationship

In [3]:
# Diagramming Data & Ouput location
splines_type = "spline"
filename = "c1607c27-4880-407e-bce0-54337045b042"

output_filetype = "png"
output_path = "/content/outputs"

In [17]:
# sample json object
architecture_data = {
    "diagram_name": "Context Diagram for Open Banking System",
    "actors": [
        {
          "name":"Personal Banking Customer",
          "description":"A customer of the bank, with personal bank accounts."
        },
        {
          "name":"Product Consumer",
          "description":"Consumes Banking Products"
        }
    ],
    "containers": [
        {
          "name":"API Application",
          "technology":"Java and Spring MVC",
          "description":"Provides Internet banking functionality via a JSON/HTTPS API.",
        },
        {
          "name":"Mobile App",
          "technology":"Xamarin",
          "description":"Provides a limited subset of the Internet banking functionality to customers via their mobile device.",
        }
    ],
    "databases": [
        {
          "name":"Database",
          "technology":"Oracle Database Schema",
          "description":"Stores user registration information, hashed authentication credentials, access logs, etc.",
        }
    ],
    "systems": [
        {
          "name": "Email Service",
          "description": "Used for email inbound & outbound services",
          "external": True
        }
    ],
    "relationships": [
        {
          "source": "Personal Banking Customer",
          "relationship": "Uses",
          "target": "API Application"
        },
        {
          "source": "API Application",
          "relationship": "Uses",
          "target": "Mobile App"
        },
        {
          "source": "API Application",
          "relationship": "Raises",
          "target": "Mobile App"
        },
        {
          "source": "API Application",
          "relationship": "Sends",
          "target": "Email Service"
        },
        {
          "source": "Email Service",
          "relationship": "Sends",
          "target": "API Application"
        },
        {
          "source": "Product Consumer",
          "relationship": "Uses",
          "target": "API Application"
        },
    ]
}

In [5]:
architecture_data['diagram_name'] 

'Context Diagram for Open Banking System'

In [23]:
# Data Transfer Objects for Diagram Generation
from typing import List

class ActorDTO:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def __repr__(self):
        return f"Actor(name={self.name!r}, description={self.description!r})"

class ContainerDTO:
    def __init__(self, name: str, technology: str, description: str):
        self.name = name
        self.technology = technology
        self.description = description

    def __repr__(self):
        return f"Container(name={self.name!r}, technology={self.technology!r}, description={self.description!r})"

class DatabaseDTO:
    def __init__(self, name: str, technology: str, description: str):
        self.name = name
        self.technology = technology
        self.description = description

    def __repr__(self):
        return f"Database(name={self.name!r}, technology={self.technology!r}, description={self.description!r})"

class SystemDTO:
    def __init__(self, name: str, description: str, external: bool = False):
        self.name = name
        self.description = description
        self.external = external

    def __repr__(self):
        return f"System(name={self.name!r}, description={self.description!r}, external={self.external!r})"

class RelationshipDTO:
    def __init__(self, source: str, relationship: str, target: str):
        self.source = source
        self.relationship = relationship
        self.target = target

    def __repr__(self):
        return f"Relationship(source={self.source!r}, relationship={self.relationship!r}, target={self.target!r})"

class ArchitectureData:
    def __init__(self, diagram_name: str, actors: List[ActorDTO], containers: List[ContainerDTO], databases: List[DatabaseDTO], systems: List[SystemDTO], relationships: List[RelationshipDTO]):
        self.diagram_name = diagram_name
        self.actors = actors
        self.containers = containers
        self.databases = databases
        self.systems = systems
        self.relationships = relationships

    def __repr__(self):
        return f"ArchitectureData(diagram_name={self.diagram_name!r}, actors={self.actors!r}, containers={self.containers!r}, databases={self.databases!r}, systems={self.systems!r}, relationships={self.relationships!r})"


In [18]:
# Json to object convertion
def json_to_architecture_data(json_data):
    actors = [ActorDTO(name=actor["name"], description=actor["description"]) for actor in json_data["actors"]]
    containers = [ContainerDTO(name=container["name"], technology=container["technology"], description=container["description"]) for container in json_data["containers"]]
    databases = [DatabaseDTO(name=db["name"], technology=db["technology"], description=db["description"]) for db in json_data["databases"]]
    systems = [SystemDTO(name=system["name"], description=system["description"], external=system.get("external", False)) for system in json_data["systems"]]
    relationships = [RelationshipDTO(source=relationship["source"], relationship=relationship["relationship"], target=relationship["target"]) for relationship in json_data["relationships"]]

    return ArchitectureData(diagram_name=json_data["diagram_name"], actors=actors, containers=containers, databases=databases, systems=systems, relationships=relationships)


In [19]:
architecture_data_object = json_to_architecture_data(architecture_data)

In [20]:
print(architecture_data_object)

ArchitectureData(diagram_name='Context Diagram for Open Banking System', actors=[Actor(name='Personal Banking Customer', description='A customer of the bank, with personal bank accounts.'), Actor(name='Product Consumer', description='Consumes Banking Products')], containers=[Container(name='API Application', technology='Java and Spring MVC', description='Provides Internet banking functionality via a JSON/HTTPS API.'), Container(name='Mobile App', technology='Xamarin', description='Provides a limited subset of the Internet banking functionality to customers via their mobile device.')], databases=[Database(name='Database', technology='Oracle Database Schema', description='Stores user registration information, hashed authentication credentials, access logs, etc.')], systems=[System(name='Email Service', description='Used for email inbound & outbound services', external=True)], relationships=[Relationship(source='Personal Banking Customer', relationship='Uses', target='API Application'), R

In [24]:
# Function generates a architecture diagarm and saves to disk
def generate_diagram(data: ArchitectureData, filename: str, splines_type: str):
    graph_attr = {
        "splines": splines_type,
    }

    if not os.path.exists('outputs'):
        os.makedirs('outputs')

    out_path = os.path.join('outputs', filename)

    with Diagram(data.diagram_name, direction="TB", graph_attr=graph_attr, outformat='png', filename=out_path, show=False):
        actors = {actor.name: Person(name=actor.name, description=actor.description) for actor in data.actors}

        with SystemBoundary("Open Banking System"):
            containers = {container.name: Container(name=container.name, description=container.description) for container in data.containers}
            databases = {database.name: Database(name=database.name, description=database.description) for database in data.databases}
            systems = {system.name: System(name=system.name, description=system.description) for system in data.systems}

            for relationship in data.relationships:
                source_name = relationship.source
                target_name = relationship.target
                relationship_type = relationship.relationship

                source = actors.get(source_name, None) or containers.get(source_name, None) or databases.get(source_name, None) or systems.get(source_name, None)
                target = actors.get(target_name, None) or containers.get(target_name, None) or databases.get(target_name, None) or systems.get(target_name, None)

                if source and target:
                  source >> Relationship(relationship_type) >> target


In [25]:
generate_diagram(architecture_data_object, filename, splines_type)